In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv", index_col="date", parse_dates=True)
X_test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv", index_col="date", parse_dates=True)

In [ ]:
df

In [ ]:
df["product"].unique()

In [ ]:
ohe = OneHotEncoder(sparse=False)

X = df.copy()
y = X.pop("num_sold")

dfOH = pd.DataFrame(ohe.fit_transform(X.drop("row_id", axis=1)))

dfOH.index = X.index
num_X = X.drop(X.drop("row_id", axis=1).columns, axis=1)
OH_X = pd.concat([X, dfOH], axis=1)
X = OH_X
X

In [ ]:
X = X.drop(["country", "store", "product"], axis=1)
X

In [ ]:
dfOH = pd.DataFrame(ohe.fit_transform(X_test.drop("row_id", axis=1)))

dfOH.index = X_test.index
num_X = X_test.drop(X_test.drop("row_id", axis=1).columns, axis=1)
OH_X = pd.concat([X_test, dfOH], axis=1)
X_test = OH_X
X_test = X_test.drop(["country", "store", "product"], axis=1)
X_test

In [ ]:
X = X.drop("row_id", axis=1)

rows_test = X_test["row_id"][:]
X_test = X_test.drop("row_id", axis=1)

In [ ]:
fourier = CalendarFourier(freq="W", order=4)

dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=2,
    additional_terms=[fourier],
    drop=True
)

In [ ]:
X = pd.concat([X, dp.in_sample()], axis=1)

In [ ]:
X

In [ ]:
X_test

In [ ]:
X = X.fillna(0.0)
X

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
model = XGBRegressor(n_estimators=200, max_depth=5)

model.fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_valid, model.predict(X_valid))

In [ ]:
model.fit(X, y)

In [ ]:
mean_absolute_error(y_valid, model.predict(X_valid))

In [ ]:
X_test

In [ ]:
next_step = dp.out_of_sample(steps=6570, forecast_index=X_test.index)

X_test = pd.concat([X_test, next_step], axis=1)

In [ ]:
preds = model.predict(X_test).astype(np.int32)

preds

In [ ]:
df = pd.DataFrame({
    "row_id": rows_test,
    "num_sold": preds
})

df

In [ ]:
df.to_csv("submission.csv", index=False)